## **YOLOv8 Model Training and Video Processing** ##

This notebook demonstrates how to train a YOLOv8 model with a custom dataset and use the trained model to process and annotate a video.

**Imports and Setup**

In [10]:
!pip install -r https://raw.githubusercontent.com/rissicay/redback-orion/main/Player_Tracking/greyhound_tracking/requirements.txt


import os
import cv2
from ultralytics import YOLO
import supervision as sv


os: For file and directory operations.

cv2: OpenCV library for video processing.

YOLO from ultralytics: YOLOv8 model for object detection.

supervision as sv: For object detection and annotation.

**Step 1: Train the YOLO Model**

In [11]:
def train_yolo():
    # Initialize the YOLO model with pretrained weights
    model = YOLO("yolov8n.pt")  #...use m or l  
    # Train the model with your custom dataset
    model.train(
        data="../data/greyhound_racing_dataset/data.yaml",  # path to your data configuration file
        epochs=10,         # number of training epochs
        imgsz=640,         # input image size
        batch=16,          # batch size
        name="yolov8n-greyhound"  # name of the training run
    )


train_yolo() Function:

1. Initializes the YOLO model with pre-trained weights (yolov8n.pt).

2. Trains the model using the specified dataset configuration (data.yaml), number of epochs, image size, batch size, and training run name.

Note: Ensure that data.yaml is correctly configured with paths to training and validation data.

**Step 2: Process Video using the Trained YOLO Model**

In [19]:
def process_video(video_path, output_path, model_path):
    # Load the fine-tuned YOLO model
    model = YOLO(model_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Verify the output directory and permissions
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if output_dir and not os.access(output_dir, os.W_OK):
        raise PermissionError(f"Write permission denied for the directory {output_dir}")

    assert cap.isOpened(), "Error reading video file"

    # Get video properties
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Initialize VideoWriter with a successful FourCC code
    fourcc_code = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_path, fourcc_code, fps, (w, h))

    if not video_writer.isOpened():
        raise IOError(f"Error initializing video writer with path {output_path}")

    # Custom function to filter greyhound detections
    def filter_greyhound_detections(detections):
        greyhound_class_id = 0  # Change to the appropriate class ID for greyhounds
        mask = detections.class_id == greyhound_class_id
        filtered_detections = sv.Detections(
            xyxy=detections.xyxy[mask],
            confidence=detections.confidence[mask],
            class_id=detections.class_id[mask]
        )
        return filtered_detections

    # Process video frames
    box_annotator = sv.BoxAnnotator(thickness=4)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Video frame is empty or video processing has been successfully completed.")
            break

        # Perform object detection
        results = model(frame, imgsz=1280)

        # Extract bounding boxes, confidences, and class IDs
        boxes = results[0].boxes.xyxy.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

        # Create detections object
        detections = sv.Detections(
            xyxy=boxes,
            confidence=confidences,
            class_id=class_ids
        )

        # # Filter greyhound detections
        # greyhound_detections = filter_greyhound_detections(detections)

        # Annotate the frame
        annotated_frame = box_annotator.annotate(scene=frame, detections=detections)

        # Write the processed frame to the output video
        video_writer.write(annotated_frame)

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

    print(f"Processed video saved to {output_path}")


**process_video() Function:**

1. Loads the fine-tuned YOLO model from the specified model_path.

2. Opens the input video file and checks directory permissions for saving the output.

3. Initializes VideoWriter to save the processed video.

4. filter_greyhound_detections(): Filters detections to include only greyhounds based on their class ID.

**Processes each video frame:**

1. Performs object detection.

2. Extracts and filters detections.

3. Annotates frames with bounding boxes.

4. Writes annotated frames to the output video file.

**Running the Script**

In [20]:
# Train the model (uncomment the line below to train)
# train_yolo()

# Path to your greyhound racing video
GREYHOUND_RACING_VIDEO_PATH = "../data/raw_videos/20240722NOTG06_V.mp4"
# Output path for the processed video
OUTPUT_VIDEO_PATH = "./output/greyhound_racing_output.mp4"
# Path to the fine-tuned model weights
MODEL_PATH = "./runs/detect/yolov8n-greyhound/weights/best.pt"

# Process the video using the trained model
process_video(GREYHOUND_RACING_VIDEO_PATH, OUTPUT_VIDEO_PATH, MODEL_PATH)



0: 736x1280 (no detections), 46.5ms
Speed: 4.5ms preprocess, 46.5ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 41.3ms
Speed: 4.6ms preprocess, 41.3ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 61.4ms
Speed: 2.6ms preprocess, 61.4ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 43.6ms
Speed: 2.4ms preprocess, 43.6ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 55.4ms
Speed: 2.1ms preprocess, 55.4ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 42.8ms
Speed: 2.2ms preprocess, 42.8ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 50.7ms
Speed: 2.2ms preprocess, 50.7ms inference, 0.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 (no detections), 41.0ms
Speed: 2.1ms prepr

Train the Model: Uncomment the train_yolo() call to train the model.

Process the Video: Specify paths for the input video, output video, and the fine-tuned model, and then call process_video() to process and annotate the video.